##### Risk adverse greedy goose is taken from: https://www.kaggle.com/ilialar/risk-averse-greedy-goose

#### Best Version: V28 (Scores 1101-3rd rank in public lb for a week)

In [ ]:
%%writefile risk_adverse_greedy_goose.py

from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col
import numpy as np

last_step_x,last_step_y = (0,0)

def get_nearest_cells(x,y):
    # returns all cells reachable from the current one
    result = []
    for i in (-1,+1):
        result.append(((x+i+7)%7, y))
        result.append((x, (y+i+11)%11))
    return result

def find_closest_food(table):
    # returns the first step toward the closest food item
    new_table = table.copy()
    
    updated = False
    for roll, axis, code in [
        (1, 0, 1),
        (-1, 0, 2),
        (1, 1, 3),
        (-1, 1, 4)
    ]:

        shifted_table = np.roll(table, roll, axis)
        
        if (table == -2).any() and (shifted_table[table == -2] == -3).any(): # we have found some food at the first step
            return code
        else:
            mask = np.logical_and(new_table == 0,shifted_table == -3)
            if mask.sum() > 0:
                updated = True
            new_table += code * mask
        if (table == -2).any() and shifted_table[table == -2][0] > 0: # we have found some food
            return shifted_table[table == -2][0]
        
        # else - update new reachible cells
        mask = np.logical_and(new_table == 0,shifted_table > 0)
        if mask.sum() > 0:
            updated = True
        new_table += shifted_table * mask

    # if we updated anything - continue reccurison
    if updated:
        return find_closest_food(new_table)
    # if not - return some step
    else:
        return table.max()

def agent(obs_dict, config_dict):
    """This agent always moves toward observation.food[0] but does not take advantage of board wrapping"""
    observation = Observation(obs_dict)
    configuration = Configuration(config_dict)
    player_index = observation.index
    player_goose = observation.geese[player_index]
    player_head = player_goose[0]
    player_row, player_column = row_col(player_head, configuration.columns)

    global last_step_x,last_step_y
    
    table = np.zeros((7,11))
    # 0 - emply cells
    # -1 - obstacles
    # -2 - food
    # -3 - head
    # 1,2,3,4 - reachable on the current step cell, number is the id of the first step direction
    
    legend = {
        1: 'SOUTH',
        2: 'NORTH',
        3: 'EAST',
        4: 'WEST'
    }
    
    # let's add food to the map
    for food in observation.food:
        x,y = row_col(food, configuration.columns)
        table[x,y] = -2 # food
        
    # let's add all cells that are forbidden
    for i in range(4):
        opp_goose = observation.geese[i]
        if len(opp_goose) == 0:
            continue
        for n in opp_goose[:-1]:
            x,y = row_col(n, configuration.columns)
            table[x,y] = -1 # forbidden cells
        if i != player_index:
            x,y = row_col(opp_goose[0], configuration.columns)
            possible_moves = get_nearest_cells(x,y) # head can move anywhere
            for x,y in possible_moves:
                table[x,y] = -1 # forbidden cells
        
    # let's add head position
    x,y = row_col(player_head, configuration.columns)
    table[x,y] = -3
    table[last_step_x,last_step_y] = -1
    last_step_x,last_step_y = x,y
    
    # the first step toward the nearest food
    step = int(find_closest_food(table))
    
    # if there is not available steps make random step
    if step not in [1,2,3,4]:
        step = np.random.randint(4) + 1
    
    return legend[step]

In [ ]:
%%writefile submission.py

from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col
import numpy as np
import time
import random
last_step_x,last_step_y = (0,0)

def get_player_coordinates(player,configuration):
    return [row_col(pos, configuration.columns) for pos in player]

def move_east(x,y):  return x, (y+1)%11
def move_west(x,y): return x, (y-1)%11
def move_north(x,y): return (x-1)%7,y
def move_south(x,y): return (x+1)%7,y

move_player = {
    'NORTH':move_north,
    'SOUTH':move_south,
    'EAST':move_east,
    'WEST':move_west
}

Opposite_Moves = {
        'NORTH':'SOUTH',
        'SOUTH':'NORTH',
        'EAST':'WEST',
        'WEST':'EAST'
    }

def calculate_trajectory(step_danger_matrix,position_queue,moves_queue,steps_queue,traversal_matrix):
    #BFS TO FIND MINIMUM DISTANCE TO ALL THE CELLS REACHABLE FROM CURRENT MOVE 
    """
    This doesn't take into account the actions of other's player (Improvement: To take action of other player into account)
    """
    FRONT = 0
    QUEUE_LENGTH = 1
    while(QUEUE_LENGTH>0):
        
        assert len(moves_queue)<=77, "Something is wrong"
        
        current_x,current_y = position_queue[FRONT]
        current_blocked_move = Opposite_Moves[moves_queue[FRONT]]
        current_step = steps_queue[FRONT] + 1
        for action in ['SOUTH','NORTH','EAST','WEST']:
            if action != current_blocked_move: 
                next_x,next_y = move_player[action](current_x,current_y)
                if (step_danger_matrix[next_x,next_y] < current_step) and (traversal_matrix[next_x,next_y]==0):
                    traversal_matrix[next_x,next_y] = current_step
                    position_queue.append((next_x,next_y)) 
                    moves_queue.append(action)
                    steps_queue.append(current_step)
                    QUEUE_LENGTH += 1
        
        QUEUE_LENGTH -= 1
        FRONT += 1
        
    return traversal_matrix

blocked_move = None
opponent_blocked_moves = {}
GAME_STEP = 1
Logging = True
def mat2string(matrix):
    output = ""
    for row in matrix.astype(int):
        for cell in row:
            output += str(cell)+" "
        output += '\n'
    return output

def agent(obs_dict, config_dict, depth=3):
    """This agent always moves toward observation.food[0] but does not take advantage of board wrapping"""
    
    global GAME_STEP,blocked_move, opponent_blocked_moves
    start_time = time.time()

    observation = Observation(obs_dict)
    configuration = Configuration(config_dict)
    self_player_id = observation.index
    
    if GAME_STEP==0: mode = 'w+'
    else: mode = 'a+'
        
    if Logging:
        game_logs = open(f'/kaggle/working/game_{self_player_id}_{depth}.log',mode)
        game_logs.write(f'GAME STEP:{GAME_STEP}  DEPTH:{depth} \n')
    
    try:
        Foods = {}
        # let's add food to the map
        for food in observation.food:
            x,y = row_col(food, configuration.columns)
            Foods[(x,y)] = 1
            if Logging:
                game_logs.write(f'Food:{x} {y} \n')

        board_matrix = np.zeros((7,11))
        step_danger_matrix = np.zeros((7,11))
    
        opponent_heads = []
        for player_id in range(4):
            player = observation.geese[player_id]
            if len(player)==0:
                continue
       
            player_coordinates = get_player_coordinates(player,configuration)
            head_x,head_y = player_coordinates[0]
            
            if (player_id==self_player_id) : 
                self_head_x,self_head_y = head_x,head_y
            else:
                opponent_heads.append((head_x,head_y,len(player)))
                
            add_tail = 0  
            
            for move in ['SOUTH','NORTH','EAST','WEST']:
                if move_player[move](head_x,head_y) in Foods:
                    add_tail = 1
                
            for i,(x,y) in enumerate(reversed(player_coordinates)):
                board_matrix[x,y] = player_id+1
                step_danger_matrix[x,y] = i+1 + add_tail
                
        if Logging:
            game_logs.write("Board Matrix \n")
            game_logs.write(mat2string(board_matrix))
            game_logs.write("Step Danger Matrix \n")
            game_logs.write(mat2string(step_danger_matrix))
            game_logs.write("Opponent heads \n")
            game_logs.write(str(opponent_heads)+'\n')

        blocked_positions = []
        for opp_x,opp_y,opp_len in opponent_heads:
            for move in ['SOUTH','NORTH','EAST','WEST']:
                x,y = move_player[move](opp_x,opp_y)
                blocked_positions.append((x,y,opp_len))
                
        if depth>0:
            for x,y,opp_len in blocked_positions:
#                 if step_danger_matrix[x,y] == 0:
                    step_danger_matrix[x,y] = opp_len + 1
        
        if Logging:
            game_logs.write("Board Matrix \n")
            game_logs.write(mat2string(board_matrix))
            game_logs.write("Step Danger Matrix \n")
            game_logs.write(mat2string(step_danger_matrix))
            
        # GET ALL THE POSSIBLE MOVES     
        Possible_Moves = ['SOUTH','NORTH','EAST','WEST']
        if blocked_move in Possible_Moves: Possible_Moves.remove(blocked_move)

        Traversal_matrices = {}

    #     return random.choice(Possible_Moves)

        for first_move in Possible_Moves:
            first_x, first_y = move_player[first_move](self_head_x,self_head_y)
            if step_danger_matrix[first_x,first_y]<=1:
                for second_move in ['SOUTH','NORTH','EAST','WEST']:
                    if Opposite_Moves[first_move] != second_move :
                        second_x,second_y = move_player[second_move](first_x,first_y)
                        if step_danger_matrix[second_x,second_y]<=2:
                            traversal_matrix = np.zeros((7,11)).astype(int)
                            traversal_matrix[first_x,first_y] = 1
                            traversal_matrix[second_x,second_y] = 2
                            position_queue = [(second_x,second_y)]
                            moves_queue = [second_move]
                            steps_queue = [2]
                            Traversal_matrices[(first_move,second_move)] = calculate_trajectory(step_danger_matrix,position_queue,moves_queue,
                                                                                                steps_queue,traversal_matrix)
                            if Traversal_matrices[(first_move,second_move)].sum()==0: return 'NORTH'
                            if Logging:
                                game_logs.write(f"First Move:{first_move} Second Move:{second_move}\n")
                                game_logs.write(f"Second x:{second_x} Second y:{second_y}\n")
                                game_logs.write(mat2string(Traversal_matrices[(first_move,second_move)]))


        # Find All Safe Positions
        """
        Heuristics: A Player is safe if there exists a cyclic path to player's current position
        """
        moves_safety = {move:0 for move in Possible_Moves}

        for first_move,second_move in Traversal_matrices:
            if Traversal_matrices[(first_move,second_move)][self_head_x,self_head_y]>0:
                moves_safety[first_move] += 1
        
        moves_safety = {k:min(2,moves_safety[k]) for k in moves_safety}

        max_safety = max(moves_safety.values())
        
        if Logging:
            game_logs.write(str(moves_safety)+"\n")
            game_logs.write(f"Max Safety: {max_safety}\n")

        # Find Positions Closest To Food
        shortest_step_to_food = None
        shortest_step = 1000
        for food_x,food_y in Foods:
            if Logging:
                game_logs.write(f"Shortest Path To Food: {food_x} {food_y}\n")
            for first_move,second_move in Traversal_matrices:
                if moves_safety[first_move]==max_safety :
                    safety_condition1 = Traversal_matrices[(first_move,second_move)][self_head_x,self_head_y]>0
                    safety_condition2 = (Traversal_matrices[(first_move,second_move)]>0).sum()>20
                    safety_condition3 = (Traversal_matrices[(first_move,second_move)]>0).sum()>10
                    if depth == 3: safety_condition = safety_condition1 and safety_condition2
                    elif depth==2 or depth==0: safety_condition = safety_condition1
                    else: safety_condition = safety_condition2
                    if safety_condition:
                        if Logging:
                            game_logs.write(f"{first_move} {second_move} : {Traversal_matrices[(first_move,second_move)][food_x,food_y]}\n")
                        if Traversal_matrices[(first_move,second_move)][food_x,food_y]<shortest_step:
                            shortest_step = Traversal_matrices[(first_move,second_move)][food_x,food_y]
                            shortest_step_to_food = first_move


        best_move = shortest_step_to_food

        if Logging:
            game_logs.write(f"Best Move: {best_move} \n")
            game_logs.write(f"Total Time Taken for the step: {time.time()-start_time}\n")
            game_logs.write("\n\n")
            game_logs.close()
    except Exception as e:
        if Logging:
            game_logs.write("Exception Raised: "+str(e))
            game_logs.close()
            assert 1==2
        best_move=None
    if best_move==None and depth>0:
        best_move = agent(obs_dict, config_dict, depth-1)
    elif best_move==None and depth==0:
        best_move = random.choice(Possible_Moves)
        
    if depth==3: 
        blocked_move = Opposite_Moves[best_move]
        GAME_STEP += 1
    return best_move

In [ ]:
!rm game*.log

In [ ]:
from kaggle_environments import evaluate, make
env = make("hungry_geese")

In [ ]:
env.reset()
env.run(['risk_adverse_greedy_goose.py', 'risk_adverse_greedy_goose.py', 'submission.py', 'risk_adverse_greedy_goose.py'])
env.render(mode="ipython", width=800, height=700)

In [ ]:
!ls

In [ ]:
def read_log(player_id,depth):
    try:
        f = open(f'game_{player_id}_{depth}.log','r+')
        text = f.read()
        text = {int(x.split(' ',1)[0]):x.split(' ',1)[1].strip() for x in text.split('GAME STEP:') if x != ''}
        return text
    except FileNotFoundError:
        return {0:''}